In [1]:
import sys,time

string = """ S how
 C inemas
 I n
 M y
 P lace
 L isting
 E verything"""

typing_speed = 390

def slow_type(t):
    for l in t:
        sys.stdout.write(l)
        sys.stdout.flush()
        time.sleep(10.0/typing_speed)
    print('\n')

print('')
slow_type(string)


 S how
 C inemas
 I n
 M y
 P lace
 L isting
 E verything



In [2]:
import requests
import sys
import emoji
from datetime import datetime as dt
from datetime import timedelta
        
def cinemoji(cinema, place):
    d_cinema = get_dict(cinema)
    
    name = d_cinema['name']
    emoticon = emoji.emojize(d_cinema['emoji'])
    
    if place == 'before':
        return emoticon +' '+ name
    elif place == 'after':
        return name +' '+ emoticon
    elif place == 'both':
        return emoticon +' '+ name +' '+ emoticon

In [3]:
print(emoji.emojize(":popcorn:"), 'WOKI', emoji.emojize(":popcorn:"))

print(emoji.emojize(":bread:"), 'Brotfabrik', emoji.emojize(":bread:"))

print(emoji.emojize(":T-Rex:"), 'Rex', emoji.emojize(":T-Rex:"))

print(emoji.emojize(":NEW_button:"), 'Neue Filmbühne', emoji.emojize(":NEW_button:"))

🍿 WOKI 🍿
🍞 Brotfabrik 🍞
🦖 Rex 🦖
🆕 Neue Filmbühne 🆕


In [3]:
import re

In [61]:
import scimple_functions as SF

imported = []
p_cinema = []
for i in SF.cinema_alias:
    p_cinema.append(i[0])

p_cinema = list(set([SF.is_cinema_alias(p) for p in p_cinema]))

for p in p_cinema:
    desc = 'importing ' + p + '...'
    end = p + ' done.'
    loader = SF.Loader(desc, end).start()
    
    imported.append(SF.import_cinema_errorless(p))
    
    loader.stop()

kinopolis done.                                                                 
rex done.                                                                       
sternlichtspiele done.                                                          
woki done.                                                                      
capitol kinocenter done.                                                        
brotfabrik done.                                                                
filmbuehne done.                                                                


In [ ]:
for i_index, imp in enumerate(imported):
    for c_i, cinema in enumerate(imp['event']):
        if cinema['timestamp'] == None:
            print(cinema)

In [70]:
['abc', None]

['abc', None]

In [72]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt
from datetime import timedelta
import locale
from bs4 import BeautifulSoup
import scimple_functions as SF

locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')

def None_AttributeError_cnt(function):
    try:
        return function.contents[0]
    except:
        return None

def kinopolis_week(url, url_week):
    result_event = []
    
    html = SF.open_url_errorless(url, url_week).read().decode("utf-8")
    
    soup = BeautifulSoup(html, "html.parser")
    match_results = soup.find_all("section", {"class": "bg-2 movie mb-2"})
    for program in match_results:
        title = program.find("a", {"class": "hl-link", "href": True}).contents[0]

        subtitle = program.find("span", {"class": "hl--sub"})
        if subtitle and len(subtitle.contents) > 0:
            subtitle = subtitle.contents[0]
        else:
            subtitle = None

        jspec = program.find_all("div", {"class": "movie__specs-el"})
        all_spec = []
        for i in jspec:
            if 'FSK' in str(i):
                all_spec.append(str(i).split('</span> ', 1)[1].split('<', 1)[0])

            elif 'Dauer' in str(i):
                runtime = str(i).split('</span> ', 1)[1].split('<', 1)[0].replace('Minuten', '').replace(' ', '')

        spec_caption = []
        caption_wrap = program.find_all("div", {"class": "caption__wrap"})
        if caption_wrap:
            for caption in caption_wrap:
                tmp = caption.find("b")
                if tmp:
                    caption_tag = tmp.contents[0]
                    caption_text = caption.find("div", {"class": "caption__text"}).contents[0].replace(' - Atmos', '')

                    for t in [title, subtitle]:
                        caption_text = "".join(caption_text.split(t)).lstrip().rstrip()

                    if caption_tag in caption_text:
                        caption_text = caption_text.split(caption_tag)[1].replace(':', '')

                    if [caption_tag, caption_text] not in spec_caption:
                        spec_caption.append([caption_tag, caption_text])

                else:
                    string = [c.string for c in caption if c.string not in [None, '\n']]
                    if len(string) == 1:
                        string = string[0]
                    else:
                        string = ': '.join([c.string for c in caption if c != '\n'])
                    spec_caption.append(['tag_all', string])


        l_day = []
        l_price = []

        table = program.find_all("div", {"class": "slider slider-6 prog-nav"})
        pattern = r'data-performance-ids=".*?</div></div></div>'
        for slider in re.findall(pattern, str(table)):
            day = re.search('<div class="prog-nav__day">(.*?)</div>', slider).group(1)

            if 'Heute' in day:
                if dt.today().hour <= 1:
                    day_date = dt.today() + timedelta(-1)
                else:
                    day_date = dt.today()

            elif 'Morgen' in day:
                if dt.today().hour <= 1:
                    day_date = dt.today()
                else:
                    day_date = dt.today() + timedelta(+1)

            else:
                day, month = day.split(' ')[-1].split('.')[:2]
                year = dt.now().year
                if int(month) < dt.now().month:
                    year += 1
                day_date = dt.strptime(day+'.'+month+'.'+str(year), '%d.%m.%Y')

            l_day.append(day_date.replace(second=0, microsecond=0))


        i = -1
        for day_slide in program.find_all("div", {"class": "prog-day__wrapper"}):
            i += 1
            day = l_day[i]

            for screen in day_slide.find_all("div", {"class": "prog2__cont", "data-performance-id": True}):            
                time_expired = None_AttributeError_cnt(screen.find("div", {"class": "prog2__time"}))
                time_alive = None_AttributeError_cnt(screen.find("a", {"class": "prog2__time time__buy_btn"}))
                if time_expired:
                    time = time_expired
                else:
                    time = time_alive

                if time:
                    hour, minute = time.split(':', 1)
                    timestamp = day.replace(hour=int(hour), minute=int(minute))

                else:
                    timestamp = None

                try:
                    room = re.search('<div>(.*?)</div>', str(screen)).group(1)
                except:
                    room = None

                seat = None_AttributeError_cnt(screen.find("div", {"class": "prog2__seats"}))

                load = None_AttributeError_cnt(screen.find("div", {"class": "prog2__scale scale-100"}))

                try:
                    tmp = screen.find("a", {"data-version": True, "href": True})['data-version'].replace('[', '').replace(']', '')
                    tmp = tmp.replace('"', '').upper().split(',')

                    if '2D' in tmp:
                        tmp.remove('2D')

                    if all(i in tmp for i in ['OV', 'OMU']):
                        tmp.remove('OV')

                    spec = []                
                    [spec.append(i) for i in tmp if i not in all_spec + spec]

                    for caption in spec_caption:
                        if caption[0] == 'tag_all':
                            all_spec.append(caption[1])

                        elif any(caption[0].lower() == t.lower() for t in spec):
                            spec = [caption[0] +': '+ caption[1] if s.lower() == caption[0].lower() else s for s in spec]

                    spec = ', '.join(all_spec + spec)
                except:
                    spec = ', '.join(all_spec)

                try:
                    ticket = 'https://www.kinopolis.de' + screen.find("a", {"data-version": True, "href": True})['href']
                except:
                    ticket = None

                price = None_AttributeError_cnt(screen.find("b"))
                if price:
                    price = price + '€'


                page = {
                    "name": 'KINOPOLIS',
                    "short name": 'kinopolis',
                    "emoji": ':burrito:',
                    "timestamp": timestamp,
                    "title": title,
                    "subtitle": subtitle,
                    "room": room,
                    "seat": seat,
                    "runtime": runtime,
                    "spec": spec,
                    "location": 'KINOPOLIS',
                    "ticket": ticket,
                    "load": load,
                    "price": price
                }

                result_event.append(page)
                
    return result_event

def kinopolis():
    result = {
        "alias": ['kinopolis', 'kp'],
        "event": []
    }
    
    raw_url = "https://www.kinopolis.de/bn/programm"
    
    for week in range(8):
        if week > 0:
            url = raw_url + '/woche-' + str(week)
        else:
            url = raw_url

        for event in kinopolis_week(url, week):
            result['event'].append(event)
        
    return result

res = kinopolis()

In [7]:
title = []
title_count = []
event_dict = {
    "title": '',
    "subtitle": '',
    "count": 0,
    "cinema": [],
    "description": []
}

for cinema in imported:
    for event in cinema['event']:
        if event['title'].lower() in event_title:
            index = event_title.index(event['title'].lower())
            event_title_count[index] += 1

        elif event['subtitle'] and event['subtitle'].lower in event_title:
            index = event_title.index(event['subtitle'].lower())
            event_title_count[index] += 1
            
        else:
            event_title.append(event['title'].lower())
            event_title_count.append(1)

for i, i_title in enumerate(event_title):
    re_title = re.sub(r"[^A-Za-z0-9]+", '', i_title)

    for j, j_title in enumerate(title):
        if re_title in re.sub(r"[^A-Za-z0-9]+", '', j_title):
            print(re_title)
        else:
            title.append(i_title)
            title_count.append(event_title_count[i])

for i in range(len(title)):
    print(title_count[i], title[i])

In [73]:
s = ['deadpool & wolverine', 'deadpool + wolverine', 'der große diktator (best of cinema) 1', 'der große diktator – best of cinema 1']
t = 'deadpool wolverine'
print(s.index(item) if 'a' in s else False)
#print(title_match(t, s))
if not None:
    print('hi')
else:
    print('ih')

print('asd'.split('-'))
s = ['a', 'b', 'c']
t = ['b', 'e', 'f']

if any(i in s for i in t):
    print('hi')

False
hi
['asd']
hi


In [239]:
def remove_minus_preview(string):
    title = string.rsplit('-', 1)
    if len(title) == 2 and 'preview' in title[-1]:
        return ' '.join(title[0].split())

    title = string.rsplit('–', 1)
    if len(title) == 2 and 'preview' in title[-1]:
        return ' '.join(title[0].split())

    return string

l = 'woodwalkers – happy family preview'
remove_minus_preview(l)
s = 'abcdeiiv'
sa = 'joker: foile a deux i + ii'
print(sa[0:])
print(' '.join(re.sub(r"[^A-ZÄÖÜa-zäöüß0-9 ]+", '', sa).split()))
clearing = sa
for i, s in enumerate(sa.split(' ')):
    if all([char in L_ROMAN for char in s]):
        clearing = ' '.join(clearing.split(' ')[:i]) +' '+ str(roman2decimal(s)) +' '+ ' '.join(clearing.split(' ')[i+1:])

print(clearing)

joker: foile a deux i + ii
joker foile a deux i ii
joker: foile a deux 1 + 2 


In [242]:
L_ROMAN = ['i', 'v', 'x', 'l', 'c', 'd', 'm']

def roman_value(r):
    if r == 'I':
        return 1
    if r == 'V':
        return 5
    if r == 'X':
        return 10
    if r == 'L':
        return 50
    if r == 'C':
        return 100
    if r == 'D':
        return 500
    if r == 'M':
        return 1000
    return -1


def roman2decimal(s):
    res = 0
    i = 0
    while i < len(s):
        s1 = roman_value(s[i].capitalize())
        if i + 1 < len(s):
            s2 = roman_value(s[i + 1].capitalize())

            if s1 >= s2:
                res += s1
            else:
                res += (s2 - s1)
                i += 1
        else:
            res += s1
        i += 1

    return res


def only_alphabet(string):
    return ' '.join(re.sub(r"[^A-ZÄÖÜa-zäöüß0-9 ]+", '', string).split())


def remove_minus_preview(l):
    result = l
    for string in l:
        if any(minus in string for minus in ['-', '–']):
            for minus in ['-', '–']:
                title = string.rsplit(minus, 1)
                if len(title) == 2 and 'preview' in title[-1]:
                    result += [' '.join(title[0].split())]

    return result


def remove_buzz(l):
    result = l
    for string in l + [only_alphabet(pos) for pos in l]:
        clearing = string
        for buzz in ['df', 'eng', 'ov', 'omu']:
            if buzz in string:
                pre_result = clearing.replace(buzz, '')
                if pre_result[0] == ' ':
                    pre_result = pre_result[1:]
                elif pre_result[-1] == ' ':
                    pre_result = pre_result[:-1]
                result += [pre_result]

    return result


def remove_roman(l):
    result = l
    for string in l + [only_alphabet(pos) for pos in l]:
        clearing = string
        for i, s in enumerate(string.split(' ')):
            if all([char in L_ROMAN for char in s]):
                clearing = ' '.join(clearing.split(' ')[:i]) + ' '
                clearing += str(roman2decimal(s)) + ' '
                clearing += ' '.join(clearing.split(' ')[i+1:])
                result += [clearing]
    if any('df' in ll.lower() for ll in l):
        print('roman', result)
    return result


def title_match(event_title, l_title):
    """
    e_title: (event[title], event[subtitle])
    l_title: list of all list(sub/titles)
    """
    for i_titlelist, titlelist in enumerate(l_title):
        match_plain = any(t in titlelist for t in event_title)
        if match_plain:
            return i_titlelist, l_title

        t_alphabet = [only_alphabet(title) for title in event_title]
        l_alphabet = [only_alphabet(item) for item in titlelist]
        match_alphabet = any(t in l_alphabet for t in t_alphabet)
        if match_alphabet:
            return i_titlelist, l_title

    return False, l_title


l_title = []
l_title_count = []
l_event = []

for cinema in imported:
    for event in cinema['event']:
        #make here a list of title, subtitle and all possible like without minus, without df
        possible_title = [event['title'].lower()]
        if event['subtitle']:
            possible_title.append(event['subtitle'].lower())

        possible_title = remove_minus_preview(possible_title)
        possible_title = remove_buzz(possible_title)
        possible_title = remove_roman(possible_title)

        index, l_title = title_match(possible_title, l_title)
        # if 'joker' in event['title'].lower():
        #     print(l_title)
        if index is not False:
            l_title_count[index] += 1

            d_event = list(set(l_event[index]['title'] + possible_title))
            l_event[index]['title'] = d_event
            l_title[index] = d_event

        else:
            d_title = [event['title'].lower()]
            if event['subtitle']:
                d_title.append(event['subtitle'].lower())

            l_title.append(d_title)
            l_title_count.append(1)

            event_dict = {
                "title": d_title,
                "count": 1,
                "cinema": [event['location'],],
                "description": [event['spec'],]
            }
            l_event.append(event_dict)

a, b = (list(x) for x in zip(*sorted(zip(l_title, l_title_count), key=lambda pair: pair[0])))
for i, aa in enumerate(a):
    print(aa, '|count:', b[i])

roman ['df: joker 1 + 2', ': joker 1 + 2', 'joker 1 2']
roman ['df: ov joker i + ii', ': ov joker i + ii', 'df:  joker i + ii', 'ov joker i ii', 'df  joker i ii', 'df: ov joker 1 ', 'df: ov joker 1  2 ', ': ov joker 1 ', ': ov joker 1  2 ', 'df: 0 ', 'df: 0  1 ', 'df: 0  1  2 ', 'ov joker 1 ', 'ov joker 1 2 ', 'df 0 ', 'df 0  1 ', 'df 0  1 2 ', 'df ov joker 1 ', 'df ov joker 1 2 ', 'ov joker 1 ', 'ov joker 1 2 ', 'df joker 1 ', 'df joker 1 2 ', 'ov joker 1 ', 'ov joker 1 2 ', 'df joker 1 ', 'df joker 1 2 ']
['abenteuerreiter - feuerland bis alaska (live reportage)', 'abenteuerreiter - feuerland bis alaska'] |count: 1
['afraid', 'they listen'] |count: 6
['alien rlus', 'alien: romulus', 'alien: rlus'] |count: 9
['alles fifty fifty', 'fifty fifty - eine erziehungskomödie'] |count: 3
['alles steht kopf 2', 'inside out 2'] |count: 16
['alter weißer mann – cinelady preview'] |count: 1
['am i ok?'] |count: 1
['architecton'] |count: 1
['beetlejuice beetlejuice'] |count: 48
['berliner philharmo

In [ ]:
event_title = []

for cinema in imported:
    for event in cinema['event']:
        if event['title'].lower() in event_title:
            index = event_title.index(event['title'].lower())
            event_title[index].append(event['title'].lower())

        elif event['subtitle'] and event['subtitle'].lower in event_title:
            index = event_title.index(event['subtitle'].lower())
            event_title[index].append(event['subtitle'].lower())
            
        else:
            event_dict = {
                "title": event['title'].lower(),
                "cinema": event['name'],
                "count": 1,
                "timestamp": event['timestamp']
            }
            event_title.append(event_dict)

for i in range(len(event_title_count)):
    print(event_title[i], event_title_count[i])

In [ ]:
import pyshorteners
type_tiny = pyshorteners.Shortener()
short_url = type_tiny.tinyurl.short(event_d['ticket'])
print(return_color(short_url, '39'))

In [ ]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt


timestamp = []
title = []
room = []
spec = []
location = []
ticket = []

url = 'https://www.imdb.com/title/tt11198810/'
page = urlopen(url)
html = page.read().decode("utf-8")

soup = BeautifulSoup(html, "html.parser")
print(soup)

#match_results = 
#jstring = json.loads(match_results)
#print(json.dumps(jstring, indent=3))
#pattern = r'<div class="relative">.*?</div>'
#match_results = re.findall(pattern, html, re.DOTALL)

In [20]:
woki = SF.import_cinema_errorless('woki')

In [31]:
import pyshorteners
import threading
import concurrent.futures

In [ ]:
for i in woki['event']:
    shortener(i['ticket'])

In [47]:
def shortener(url):
    type_tiny = pyshorteners.Shortener()
    short_url = type_tiny.tinyurl.short(url)
    return short_url

In [ ]:
#multithreading
threads = list()

print(len(woki['event']))

for event in woki['event']:
    x = threading.Thread(target=shortener, args=(event['ticket'],))
    threads.append(x)
    x.start()

for url, thread in enumerate(threads):
    print(url, thread)
    thread.join()

In [ ]:
for event in woki['event']:
    print(event['ticket'])

In [28]:
import rottentomatoes as rt

In [ ]:
for event in woki['event']:
    while True:
        try:
            movies = rt.tomatometer(event['title'])
            print(movies)

        except Exception as e:
            print("\r" , end="", flush=True)
            
            print('retrying..')
            time.sleep(4)

In [63]:
movie = ia.search_movie('harry potter')[0]
print(movie)
movie.infoset2keys

Harry Potter and the Sorcerer's Stone


{}

In [12]:
from imdb import Cinemagoer

In [14]:
ia = Cinemagoer()

In [ ]:
for event in woki['event']:
    while True:
        try:
            movies = ia.search_movie(event['title'])
            print(movies)

        except Exception as e:
            print("\r" , end="", flush=True)
            
            print('retrying..')
            time.sleep(4)

In [19]:
# create an instance of the Cinemagoer class
for director in the_matrix['directors']:
    print(director['name'])
print(sorted(the_matrix.keys()))
print(ia.get_movie_infoset())
ia.update(the_matrix, ['technical'])
print(the_matrix.infoset2keys['technical'])
print(the_matrix.get('tech'))

Lana Wachowski
Lilly Wachowski
['akas', 'animation department', 'art department', 'art direction', 'aspect ratio', 'assistant director', 'box office', 'camera and electrical department', 'canonical title', 'cast', 'casting department', 'casting director', 'certificates', 'cinematographer', 'color info', 'composer', 'costume department', 'costume designer', 'countries', 'country codes', 'cover url', 'director', 'distributors', 'editor', 'editorial department', 'full-size cover url', 'genres', 'imdbID', 'kind', 'language codes', 'languages', 'localized title', 'location management', 'long imdb canonical title', 'long imdb title', 'make up', 'miscellaneous crew', 'music department', 'original air date', 'original title', 'other companies', 'plot', 'plot outline', 'producer', 'production companies', 'production design', 'production manager', 'rating', 'runtimes', 'script department', 'set decoration', 'smart canonical title', 'smart long imdb canonical title', 'sound crew', 'sound mix', 's

In [14]:
import time
ia = Cinemagoer()

for i in movietitle[6:10]:
    print('##', i)

    before = time.time()
    movie = ia.search_movie(i)[0].movieID
    print(movie)
    print(time.time()-before)
    before = time.time()
    movies = ia.get_movie(movie, info=['rating'])
    #movie = ia.get_movie(movie, info=['cast'])
    #print(movie.infoset2keys)
    #print(ia.get_movie(movie, info=['rating']))
    print(movies.get('rating'))
    movies = ia.get_movie(movie)
    print(movies.infoset2keys)
    print(time.time()-before)
    before = time.time()


## the nun 2


2023-10-25 01:48:24,913 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


10160976
1.3201336860656738
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'akas', 'production companies', 'distributors', 'special effects companies', 'other companies'], 'plot': ['plot', 'synopsi

2023-10-25 01:48:28,576 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


28627002
0.8141398429870605
None
{'main': ['localized title', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'certificates', 'rating', 'votes', 'cover url', 'imdbID', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'production companies'], 'plot': ['plot']}
2.3289988040924072
## barbie


2023-10-25 01:48:31,888 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


1517268
0.9829950332641602
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'animation department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'thanks', 'akas', 'production companies', 'distributors', 'special effects companies', 'other companie

2023-10-25 01:48:36,050 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


15398776
1.0211632251739502
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'animation department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'thanks', 'akas', 'top 250 rank', 'production companies', 'distributors', 'special ef

In [ ]:
import time
ia = Cinemagoer()
print(ia.get_movie_infoset())

for i in movietitle:
    print('##', i)

    before = time.time()
    movie = ia.search_movie(i)[0]
    print(movie)
    print('found')
    print(time.time()-before)
    before = time.time()

    ia.update(movie, info=['rating'])
    print(time.time()-before)
    before = time.time()

    print('info', movie.current_info)
    print(time.time()-before)
    before = time.time()

    print(type(movie))

    print(time.time()-before)
    before = time.time()

    try:
        print(movie.infoset2keys)
    except KeyError:
        print('this')
    print('\n')

In [44]:
try:
    print(movie.infoset2keys)
except KeyError:
    print('this')
print('\n')

{}




In [35]:
brot = bf.brotfabrik()

ia = Cinemagoer()
print(ia.get_movie_infoset())

print(resp)

for i in movietitle:
    try:
        print('##', i)
        
        movie = ia.search_movie(i)[0]
        for key, value in movie.items():
            print(key, value)
        print('\n')
        
        rtmovie = rt.Movie(i)
        if movie['year'] == rt.year_released(i):
            print(rtmovie, '\n')
        else:
            raise KeyError
        
    except KeyError:
        try:
            ia.update(movie, info=['main'][0])
            print(movie.get('original title'))
            movie = rt.Movie(movie.get('original title'))
            print(movie, '\n')
        except:
            print('no ratings on tomatoes')
            continue
        continue

['airing', 'akas', 'alternate versions', 'awards', 'connections', 'crazy credits', 'critic reviews', 'episodes', 'external reviews', 'external sites', 'faqs', 'full credits', 'goofs', 'keywords', 'list', 'locations', 'main', 'misc sites', 'news', 'official sites', 'parents guide', 'photo sites', 'plot', 'quotes', 'recommendations', 'release dates', 'release info', 'reviews', 'sound clips', 'soundtrack', 'synopsis', 'taglines', 'technical', 'trivia', 'tv schedule', 'video clips', 'vote details']


NameError: name 'resp' is not defined

In [171]:
brot = bf.brotfabrik()

ia = Cinemagoer()
print(ia.get_movie_infoset())

for i in brot['title']:
    print(i)
    movie = ia.search_movie(i)
    for m in movie:
        print(m['title'], type(m))

        for key, value in m.items() :
            print (key, value)

        print('\n')

NameError: name 'bf' is not defined

In [ ]:
ia = Cinemagoer()
print(ia.get_movie_infoset())

movie = ia.search_movie('rose')

print(movie, type(movie[2]))
for m in movie:
    print(m['title'], type(m))
    
    for key, value in m.items() :
        print (key, value)
        
    print('\n')

In [5]:
# create an instance of the Cinemagoer class
ia = Cinemagoer()

# get a movie
movie = ia.get_movie('0133093')

# print the names of the directors of the movie
print('Directors:')
for director in movie['directors']:
    print(director['name'])

# print the genres of the movie
print('Genres:')
for genre in movie['genres']:
    print(genre)

# search for a person name
people = ia.search_person('Mel Gibson')
for person in people:
    print(person.personID, person['name'])

Directors:
Lana Wachowski
Lilly Wachowski
Genres:
Action
Sci-Fi
0000154 Mel Gibson
12550288 Mel Gibson
0317112 Mel Gibson
2153167 Mel Gibson
15271356 Mel Gibson
4798507 Mel A. Gibson
3323885 Adam Taylor
2800055 Mel Gibson
3742421 Melina Mari Bryant
0317096 Meg Gibson
0879085 Tyrese Gibson
0684545 Julie Pinson
8908771 Millie Gibson
0316995 Hoot Gibson
0316945 Donal Gibson
4047961 Noel Gibson
6521923 Mel Gibbon
2988370 Mal Gibson
0316932 Dean Paul Gibson
0316878 Cal Gibson


In [36]:
for i in movietitle:
    print('-'*48)
    movieinput = i

    resp = requests.get('https://rotten-tomatoes-api.ue.r.appspot.com/search/' + movieinput)

    print(resp.json(), '\n')
    print(len(resp.json().get('movies')))

    i = 0
    if len(resp.json().get('movies')) > 1:
        for m in resp.json().get('movies'):
            i += 1
            print(f'{i:2.0f}.', m.get('name'))
            print(' '*3, 'release:', m.get('year'))

        hit = int(abort(input('-> select wich file: ')))
        print(resp.json().get('movies')[hit - 1])

    else:
        print(resp.json().get('movies')[0].get('name'))


------------------------------------------------
{'movies': [{'name': 'Killers of the Flower Moon', 'tomatometer': 92, 'audience_score': 85, 'weighted_score': 89, 'genres': ['Crime', 'Drama'], 'rating': 'R', 'duration': '3h 26m', 'year': '2023', 'actors': ['Leonardo DiCaprio', 'Robert De Niro', 'Jesse Plemons', 'Lily Gladstone', 'Tantoo Cardinal'], 'directors': ['Martin Scorsese']}]} 

1
Killers of the Flower Moon
------------------------------------------------
{'movies': []} 

0


IndexError: list index out of range

In [244]:
import time
def import_url(url):
    while True:
        try:
            page = urlopen(url)

        except Exception as e:
            print("\r" , end="", flush=True)
            
            print(e)
            time.sleep(4)
        else:
            print('finally.')
            return page
        

In [8]:
url =         
import_url(url)

In [ ]:
soup = BeautifulSoup(html, 'html5lib')
print(soup.prettify())

In [ ]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt
from datetime import timedelta
from bs4 import BeautifulSoup

url = "https://www.kinopolis.de/bn/programm"
html = urlopen(url).read().decode("utf-8")

pattern = r'<section class="bg.*?<section class="bg'
match_results = re.findall(pattern, html, re.DOTALL)

In [ ]:
def ():
    result = {
        "alias": [],
        "event"
    }

In [ ]:
<section class="bg-2  movie mb-2"  data-fsk="12" data-genre='["komödie"]' data-version='["2d"]'>

In [59]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt
from datetime import timedelta
from bs4 import BeautifulSoup

url = "https://www.capitol-siegburg.de/programm"
html = urlopen(url).read().decode("utf-8")

In [95]:
pattern = r'<script type="application.*?</script>'
#match_results = re.search(pattern, html, re.DOTALL)
match_results = re.search(r'json">\[.*?\]<', html, re.DOTALL)[0][6:-1]

for match in json.loads(match_results)[:1]:
    title = match['workPresented']['name']
    
    t_start = match['startDate']
    t_end = match['endDate']

    spec = match['videoFormat']

    print(title)
    print(match)

Das Geheimnis von la Mancha
{'@context': 'http://schema.org', '@type': 'ScreeningEvent', 'name': 'Das Geheimnis von la Mancha 2D', 'startDate': '2024-04-28T14:30:00+02:00', 'endDate': '2024-04-28T15:57:00+02:00', 'location': {'@type': 'MovieTheater', '@id': 'https://www.capitol-siegburg.de#MovieTheater', 'name': 'Capitol Kinocenter', 'image': 'https://www.capitol-siegburg.de/clients/119/img/logo.png', 'telephone': '(0 22 41) 6 22 88', 'priceRange': '6 - 12 €', 'address': {'@type': 'PostalAddress', 'streetAddress': 'Augustastraße 20', 'addressLocality': 'Siegburg', 'postalCode': '53721'}}, 'image': 'https://www.capitol-siegburg.de/images/Breite_235px_RGB/p_110804.jpg', 'performer': 'screening only', 'workPresented': {'@type': 'Movie', '@id': 'https://www.capitol-siegburg.de#110804', 'name': 'Das Geheimnis von la Mancha', 'image': 'http://cineprog.de/images/Breite_235px_RGB/p_110804.jpg', 'description': 'Alfonso ist der Nachfahre des legendären Don Quijote. Gemeinsam mit Pancho und Victo

In [163]:
from urllib.request import urlopen
import re
import json
from datetime import datetime as dt


l_spec = []

ignore_value = ['{}', '', 'idf']


result = {
    "alias": ['Capitol Kinocenter', 'cs', 'capitol', 'capitol siegburg'],
    "event": []
}

pattern = r'var programm = .*?; var wallpaper'
sub_match_results = re.search(pattern, html, re.DOTALL)[0][15:-15]
match_results = json.loads(sub_match_results)['filme']

for _, film in match_results.items():
    film_fakten = film['filmfakten']
    
    l_spec.append(film_fakten['Versionsmarker'])

    titel = film_fakten['titel']

    runtime = film_fakten['laufzeit']

    vorst_fakten = film['vorstellungen']

    for key, value in vorst_fakten['vorstellungen_fakten'].items():
        if str(value) not in ignore_value and key not in ignore_value:
            if value == '1':
                l_spec.append(key.capitalize())
            else:
                l_spec.append(str(key) +': '+ str(value))

    for _, info in vorst_fakten['termine'].items():
        if isinstance(info, dict):
            info = [info]

        for termin in info:
            time = termin['datum'] +'-'+ termin['zeit']
            timestamp = dt.strptime(time, '%Y-%m-%d-%H:%M')

            room = termin['saal']

            ticket = termin['link_desktop']

            spec = ', '.join(l_spec)


            page = {
                "name": 'Capitol Kinocenter',
                "short name": 'capitol',
                "emoji": ':film_frames:',
                "timestamp": timestamp,
                "title": title,
                "subtitle": None,
                "room": room,
                "seat": None,
                "runtime": runtime,
                "spec": spec,
                "location": 'Capitol Kinocenter',
                "ticket": ticket,
                "load": None,
                "price": None
            }

            result['event'].append(page)

print(result['alias'])

['Capitol Kinocenter', 'cs', 'capitol', 'capitol siegburg']


In [48]:
dt.now().year

2024

In [53]:
t = 10
0 < t < 10

False